In [36]:
%load_ext autoreload
from config import Marco,content_generator
import pandas as pd
import os 
from utils import get_selected_cols_df,get_selected_values_df,joinStr,ensure_dir,get_daily_report_df,getDate_in_month,get_selected_values_over_df,greeting,get_night_data
from datetime import datetime 
from glob import glob
import warnings
import math
import numpy as np
warnings.filterwarnings("ignore")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
data_path = 'daily_route_data'
##########
AFTERHOUR = False
contact_list_path = ".\\night_shift\\*.xlsx"
main_data_path =sorted(glob(f".\\{data_path}\\*.csv"))[-1]

##########

df = pd.read_csv(main_data_path)
print(main_data_path)
print(AFTERHOUR)
str(datetime.now())[0:10]

.\daily_route_data\IOD_MONITORING_DETAIL_US_2021-06-07 17-22.csv
False


'2021-06-07'

In [38]:
main_data_path

'.\\daily_route_data\\IOD_MONITORING_DETAIL_US_2021-06-07 17-22.csv'

## Today's date

In [39]:
class DailySheet:
    def __init__(self, df, user, need_over_due = False):
        self.mode = ['end_of_month','regular'][ getDate_in_month() <= 22]
        self.alertType = user.alertType
        self.need_over_due = need_over_due
        self.df = df
        self.user = user
        self.week_day = datetime.now().weekday()
      
    def filter_by_carriers(self,df):
        return get_selected_values_df(df,self.user.carriers_code,col_name = 'Carrier Code')
    
    
    def filter_by_allert(self,df):
        self.all_allert_df = get_selected_values_df(df,self.alertType, col_name = 'Alert Type')
        if self.mode == 'regular':
            return get_selected_values_df(df,self.alertType[:5], col_name = 'Alert Type')
        elif self.mode == 'end_of_month':
            return get_selected_values_df(df,self.alertType, col_name = 'Alert Type')
    
    
    def get_each_carriers_iod_email_content(self , cols_send2carrier):
        user_carrier_df = self.filter_by_carriers(self.df)
        past_alert_user_carrier_df = self.filter_by_allert(user_carrier_df)
        
        ## dont show too much information to the carriers
        self.cleaned_df = past_alert_user_carrier_df
        return get_selected_cols_df(past_alert_user_carrier_df, cols_send2carrier)
    
    
    def get_high_value_loads(self):
        high_values_df = get_selected_values_over_df(self.cleaned_df, 50000.00, col_name = 'Sales')
        return high_values_df        
    
    
    def get_overdue(self,df):
        return get_selected_values_df(self.filter_by_carriers(df),['Overdue 4', 'Overdue 3', 'Overdue 2','Overdue 1'],col_name = 'Alert Type')
    

    def save_daily_content2carriers(self, df, date_path):
        unique_files = sorted(df['Carrier Code'].unique())
        timestamp = str(datetime.now())[0:10]
        date_path = os.path.join(joinStr('Carrier Code'),str(datetime.now())[0:10])
        ensure_dir(date_path)
        
        ## unique identifier
        order = glob(f".\{data_path}\*.csv")[-1][-12:-4]
        clock = "".join([d for d  in str(datetime.now())[11:16] if d.isdigit()])
        final_folder = os.path.join(date_path, clock + "_" + str(order) )
        ensure_dir(final_folder)
        
            
        ## One excel would be convienience    
        writer = pd.ExcelWriter(os.path.join(final_folder,'daily_to_carriers.xlsx'), engine='xlsxwriter')
        workbook=writer.book          
        for carrier_code in unique_files:
            emails = Marco.carrier_email.get(carrier_code)
            df_ONE_CARRIERS = df[df['Carrier Code'] == carrier_code]
            df_ONE_CARRIERS['Ship Date'] = pd.to_datetime(df_ONE_CARRIERS['Ship Date'])
            df_ONE_CARRIERS["Aging"] = df_ONE_CARRIERS['Ship Date'].apply(lambda x : (datetime.today() - x)//np.timedelta64(1,'D'))
            df_ONE_CARRIERS["High Value"] = df_ONE_CARRIERS['Sales'].apply( lambda x : f"HIGH VALUES over $ {int(math.floor(x//10000) * 10000)}, Prioritize!" if x > 20000 else "")
            df_ONE_CARRIERS = df_ONE_CARRIERS[list(df_ONE_CARRIERS.columns)[-2:] + list(df_ONE_CARRIERS.columns)[:-2]]
            df_ONE_CARRIERS.loc[0,'E-mail to'] = emails
            df_ONE_CARRIERS.loc[0,'Subject'] = f"{carrier_code} Pending IOD {timestamp}"
            df_ONE_CARRIERS.loc[0,'Content'] = content_generator(carrier_code,df_ONE_CARRIERS)
            df_ONE_CARRIERS.loc[0,'LTL'] = "For all the LTL loads I sent, I have check online ,but unluckily there is no valid update. Please advise status ASAP "
            print(f"Working on {carrier_code} !!!!!")

            worksheet=workbook.add_worksheet(carrier_code)
            writer.sheets[carrier_code] = worksheet
            if carrier_code in ['FXFE', 'FDEN']:
                df_ONE_CARRIERS.sort_values("Aging",ascending = False).to_excel(writer,sheet_name=carrier_code,startrow=1 , startcol=0,index = False)
            else:
                df_ONE_CARRIERS.sort_values("Alert Type").to_excel(writer,sheet_name=carrier_code,startrow=1 , startcol=0,index = False)
        writer.save()
            
        if self.week_day == 0 or self.need_over_due:
            over_due_df = self.get_overdue(df)
            over_due_df.to_excel(os.path.join(date_path,f"Over_due_Marco_{timestamp}.xlsx"),index = False,engine='xlsxwriter')
            print(f" Oh Monday overdue !! Please send to Jinwoo !!")                                   
                
        ## all carriers 

        df.to_excel(os.path.join(final_folder,f"all_{timestamp}.xlsx"),index = False,engine='xlsxwriter')
        print(f" Finished ALL carriers content !!")
        
        self.cleaned_df.to_excel(os.path.join(final_folder,f"all_cols_{timestamp}.xlsx"),index = False,engine='xlsxwriter')
        print(f" Finished ALL cols and carriers content !!")
        
        if self.mode == 'end_of_month':
            self.get_high_value_loads().to_excel(os.path.join(final_folder,f"high_values_{timestamp}.xlsx"),index = False,engine='xlsxwriter')
            print(f" Oh high values !! Finished high content !!")
#             
def daily_route(df,Marco):
    print(f" Today is {str(datetime.now())[0:10]}" + f" {greeting()} !!!!! ")
    pic= Marco()
#     pic.carriers_code = l
    if AFTERHOUR:
        night_carriers_code, night_carrier_email = get_night_data(contact_list_path)
        pic.carrier_email = night_carrier_email
        pic.carriers_code = night_carriers_code
#         pic.alertType = ['Overdue 4', 'Overdue 3','Overdue 1', 'Overdue 2', 'Today']

    dailysheet = DailySheet(df,pic,need_over_due=True)
    dailysheet.mode = 'end_of_month'
    
    ## prepare for the data to e-mail
    dailysheet.save_daily_content2carriers(dailysheet.get_each_carriers_iod_email_content(pic.colsSend2Carrier),data_path)

        

In [40]:
# Marco.
daily_route(df,Marco)
files = glob(f".\\{data_path}\\*.csv")

 Today is 2021-06-07 Afternoon !!!!! 
Working on AERG !!!!!
Working on AMZX !!!!!
Working on AVRT_ATL !!!!!
Working on AVRT_FTW !!!!!
Working on AVRT_ORL !!!!!
Working on AVRT_ROM !!!!!
Working on BTVP !!!!!
Working on CNWY !!!!!
Working on CVYI !!!!!
Working on HJBI !!!!!
Working on HMES !!!!!
Working on HRCF !!!!!
Working on HSXF !!!!!
Working on PASC !!!!!
Working on PGAA !!!!!
Working on RDWY_CRA !!!!!
Working on RDWY_FON !!!!!
Working on RDWY_FTW !!!!!
Working on RDWY_POR !!!!!
Working on RDWY_ROM !!!!!
Working on RETL !!!!!
Working on TFFC !!!!!
Working on TQLI_FW !!!!!
Working on UTP2 !!!!!
Working on WENP !!!!!
 Oh Monday overdue !! Please send to Jinwoo !!
 Finished ALL carriers content !!
 Finished ALL cols and carriers content !!
 Oh high values !! Finished high content !!


In [79]:

sorted(df['Carrier Code'].unique())
df[df['Carrier Code'] == "GFAD"]

,Div,Load ID,Stop Seq,Carrier Code,Carrier Name,Service Type,Container Type,Customer PO,Alert Type,Sales,...,GERP Order No,Pick Order No,Pro No,From Address,To Address,Pick Appointment,Carrier Contact E-Mail,Carrier Contact Tel,Origin Location Type,Destination Location Type


### This is for the warehouse get the BOL

In [41]:
loads_list = [
# 11637175,
# 11647290,
# 11648850,
# 11607952,
# 11639250,
# 11639806,
# 11639841,
11727467


]  
COLS = ["Customer PO","Pro No","Load ID","Carrier Code","Service Type","Sales","Ship Date","Customer Name","To Address","GERP Order No","Pick Order No"]


def get_df_to_wh(df, loads_list = loads_list, cols = COLS):
    to_warehouse_df = get_selected_values_df(df ,loads_list, col_name = 'Load ID')
    res = to_warehouse_df[COLS]
    res = res.drop_duplicates()
    print("get the data from tms for WAVE number and Purchase Number!!")
    date_path = os.path.join(joinStr('Carrier Code'),str(datetime.now())[0:10])
    res["Purchase Order No"]="pls_fill"
    res["Wave No"] = "pls_fill"
    res = res[["Purchase Order No"] + ["Wave No"] + COLS]
    res.to_excel(os.path.join(date_path,"BOL_NEED.xlsx"),index = False)
    print(res)
    
get_df_to_wh(df, loads_list,COLS)

get the data from tms for WAVE number and Purchase Number!!
  Purchase Order No   Wave No Customer PO    Pro No   Load ID Carrier Code  \
0          pls_fill  pls_fill        3194  11727467  11727467         PGAA   

  Service Type   Sales         Ship Date       Customer Name  \
0         AIR3  1213.8  05/13/2021 15:03  SNOWFLAKE GUAM CO.   

                                          To Address  GERP Order No  \
0  349 E HARMON INDUSTRIAL PARK ROAD DFS STOCK TA...     1041897302   

    Pick Order No  
0  NJ221041900101  


### This is for the aging


In [34]:
## get the lastest data and conbine it together!
tm_path = glob(".\daily_report_data\*.xls")[-1]
iod_path = glob(".\daily_report_data\*.csv")[-1]
fedex_path = glob(f"..\\fedEx\\{str(datetime.now())[0:10]}\\*.xlsx")[-1]
df_tm = pd.read_excel(tm_path)
df_iod = pd.read_csv(iod_path)
report_df = get_daily_report_df(df_iod,df_tm)




In [35]:
df_iod_marco = get_selected_values_df(df_iod,Marco.carriers_code,'Carrier Code')

In [36]:
df_iod_marco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1243 entries, 1 to 3601
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Div                          1243 non-null   object 
 1   Load ID                      1243 non-null   int64  
 2   Stop Seq                     1243 non-null   object 
 3   Carrier Code                 1243 non-null   object 
 4   Carrier Name                 1243 non-null   object 
 5   Service Type                 1243 non-null   object 
 6   Container Type               1243 non-null   object 
 7   Customer PO                  1243 non-null   object 
 8   Alert Type                   1243 non-null   object 
 9   Sales                        1243 non-null   float64
 10  Ship Date                    1243 non-null   object 
 11  Required Delivery Date       1243 non-null   object 
 12  Drop Appointment             50 non-null     object 
 13  IOD Report Date   

In [37]:
cates = ["Service Type","Stop Seq","Carrier Code"]
for col in cates:
    df_iod_marco[col]= df_iod_marco[col].astype("category")

In [38]:
df_iod_marco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1243 entries, 1 to 3601
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Div                          1243 non-null   object  
 1   Load ID                      1243 non-null   int64   
 2   Stop Seq                     1243 non-null   category
 3   Carrier Code                 1243 non-null   category
 4   Carrier Name                 1243 non-null   object  
 5   Service Type                 1243 non-null   category
 6   Container Type               1243 non-null   object  
 7   Customer PO                  1243 non-null   object  
 8   Alert Type                   1243 non-null   object  
 9   Sales                        1243 non-null   float64 
 10  Ship Date                    1243 non-null   object  
 11  Required Delivery Date       1243 non-null   object  
 12  Drop Appointment             50 non-null     object  
 13  IOD

In [39]:
daily_report = pd.pivot_table(df_iod_marco,index=["Service Type","Carrier Code"],
                              columns="Stop Seq",values=["Load ID"],aggfunc='count',
                              fill_value=0,margins=True)

daily_report.

SyntaxError: invalid syntax (<ipython-input-39-5081f6475d0b>, line 5)

In [ ]:
timestamp = str(datetime.now())[0:10]
daily_report.to_csv(os.path.join("daily_report_ouput",f"{timestamp}_daily_report.csv"))

In [ ]:
def update_iod_tms_with_dateandcoment(browers,date = '01/01/2021 23:05',string = "late ship confirmation "):
    i = 0 
    while browers.find_element_by_id(f"iodMonitoringDetailGrid_BODY_{i}_CHECKBOX"):
        checkButton = browers.find_element_by_id(f"iodMonitoringDetailGrid_BODY_{i}_CHECKBOX")
        time.sleep(0.1)
        checkButton.click()
        time.sleep(0.2)
        enter = browers.find_element_by_id(f'iodMonitoringDetailGrid_BODY_{i}_20_INPUT')
        enter.click()
        time.sleep(0.1)
        enter.send_keys(date)
        remark = browers.find_element_by_id(f'iodMonitoringDetailGrid_BODY_{i}_21_INPUT')
        remark.send_keys(string + "" + datetime.now().strftime("%m/%d/%Y") + "Marco")
        i += 1
        if i > 30:
            browers.find_element_by_tag_name('body').send_keys(Keys.END)
browers = driver
time.sleep(390)
update_iod_tms_with_dateandcoment(browers)


def update_eta_tms_with_dateandcoment(browers,date = '02/01/2021 09:30',string = "EOM FAILURE  "):
    i = 0 
    while browers.find_element_by_id(f"iodMonitoringDetailGrid_BODY_{i}_CHECKBOX"):
        checkButton = browers.find_element_by_id(f"iodMonitoringDetailGrid_BODY_{i}_CHECKBOX")
        time.sleep(0.1)
        checkButton.click()
        time.sleep(0.2)
        enter = browers.find_element_by_id(f'iodMonitoringDetailGrid_BODY_{i}_17_INPUT')
        enter.click()
        time.sleep(0.1)
        enter.send_keys(date)
        
#         T = browers.find_element_by_id(f'iodMonitoringDetailGrid_BODY_{i}_18_INPUT')
#         T.click()
#         time.sleep(0.1)
#         T.send_keys("Carrier")
        
        remark = browers.find_element_by_id(f'iodMonitoringDetailGrid_BODY_{i}_19_INPUT')
        remark.send_keys(string + "" + datetime.now().strftime("%m/%d/%Y") + "Marco")
        i += 1
        if i > 30:
            browers.find_element_by_tag_name('body').send_keys(Keys.END)
browers = driver
update_eta_tms_with_dateandcoment(browers)

In [83]:
def gen():
    
    try :
        yield 'value'
    except ValueError:
        yield "handle"
        
    finally:
        print("f")
        
x = gen()
next(x)
e = ValueError("some ")
print(x.throw(e))

f
handle


In [88]:
x.close()

f
